## Environment Setup

In [1]:
! pip install langchain_community langchain_mistralai langchainhub langchain tiktoken chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 40.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.9/617.9 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 73.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.6/278.6 kB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 90.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.8/55.8 kB 4.0 MB/s eta 0:00:00
  

In [2]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = '<your key goes here>'
os.environ['MISTRAL_API_KEY'] = '<your key goes here>'


In [3]:
questions = [ "What does Moby Dick say about humanity's struggle against nature?",
              "How does Moby Dick explore the theme of obsession through Ahab's quest?",
              "How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?",
              # "What motivates Ishmael to join the Pequod, and how does he change throughout the novel?",
              # "What does the white whale symbolize in Moby Dick, and how does it relate to Ahab's obsession?",
              # "How does the novel Moby Dick use the ocean as a symbol of the unknown?",
              # "How does Melville’s narrative style in Moby Dick contribute to the sense of adventure and mystery?",
              # "How does Ishmael’s perspective shape the reader’s understanding of the story in Moby Dick?",
              # "How does Moby Dick reflect 19th-century views on fate and destiny?",
              # "What philosophical questions does Melville raise about human existence and purpose in Moby Dick?",
              # "How does Melville use imagery to depict the sea as both beautiful and terrifying in Moby Dick?",
              # "How does Moby Dick describe the vastness and danger of the open sea?",
              # "What moral dilemmas do the crew members face in Moby Dick?",
              # "How does Moby Dick present Ahab's pursuit of revenge as both justified and self-destructive?"
              ]

question = questions[0]

## Indexing

In [4]:
from langchain_community.document_loaders import WebBaseLoader
# there can be mulitple urls
loader = WebBaseLoader("https://www.gutenberg.org/cache/epub/2701/pg2701.txt")
books = loader.load()

### Splitting the text

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=300,
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(books)
len(splits)

1412

## vectorize chunks

In [6]:
from langchain_mistralai import MistralAIEmbeddings
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents=splits,
                                    embedding=MistralAIEmbeddings())

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/langchain_mistralai/embeddings.py:169: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


## Retrieval

### Init Retriever

In [7]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 1})

### Step-Back Prompt

In [8]:
from langchain_core.prompts import ChatPromptTemplate, FewShotChatMessagePromptTemplate
generification_play = [
    {
        "input": "What are the economic impacts of tourism in Venice?",
        "output": "How does tourism affect local economies in popular cities?",
    },
    {
        "input": "What are the main causes of air pollution in New Delhi?",
        "output": "What are the common causes of air pollution in large cities?",
    },
]
# We now transform these to example messages
play_prompt_template = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}"),
        ("ai", "{output}"),
    ]
)
play_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=play_prompt_template,
    examples=generification_play,
)
stepback_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
"""
You are a knowledgeable assistant.
Your task is to transform specific questions into broader,
more general questions that are easier to answer and provide a wider perspective.
This approach is known as creating 'step-back' questions.
Here are a few examples:
""",
        ),
        # Play Prompt
        play_prompt,
        # Genuine user question
        ("user", "{question}"),
    ]
)




### Output original questions and their step-back questions

In [10]:
from langchain_mistralai import ChatMistralAI
from langchain.schema.output_parser import StrOutputParser
stepback_chain = stepback_prompt | ChatMistralAI(temperature=0) | StrOutputParser()

for question in questions:
    print("-------------")
    print("Original Question:\n\t"+question)
    stepback_question = stepback_chain.invoke({"question":question})
    print("Generic question:\n\t"+stepback_question)


-------------
Original Question:
	What does Moby Dick say about humanity's struggle against nature?
Generic question:
	How does literature portray humanity's relationship with and struggle against nature?

(Note: While Moby Dick is a rich source for exploring this theme, limiting the question to this single work might not provide as comprehensive an answer as considering the broader context of literature as a whole.)
-------------
Original Question:
	How does Moby Dick explore the theme of obsession through Ahab's quest?
Generic question:
	How does literature use the concept of obsession in character development?
-------------
Original Question:
	How is Captain Ahab portrayed as both a hero and a villain in Moby Dick?
Generic question:
	How can complex characters in literature be perceived as having both positive and negative qualities?


# Generation

In [11]:
from langchain_core.runnables import RunnableLambda


response_prompt_template = """
You are a world knowledge expert.
Answer the following question thoroughly.
Use relevant information from the provided context but disregard any irrelevant details.

Context:
{normal_context}
{step_back_context}

Question: {question}
Answer:"""
response_prompt = ChatPromptTemplate.from_template(response_prompt_template)

chain = (
    {
        # Retrieve context using the normal question
        "normal_context": RunnableLambda(lambda x: x["question"]) | retriever,
        # Retrieve context using the step-back question
        "step_back_context": stepback_chain | retriever,
        # Pass on the question
        "question": lambda x: x["question"],
    }
    | response_prompt
    | ChatMistralAI(temperature=0)
    | StrOutputParser()
)

chain.invoke({"question": question})

'Captain Ahab, the captain of the Pequod in Herman Melville\'s "Moby Dick," is portrayed as a complex character who embodies both heroic and villainous qualities.\n\nOn the one hand, Ahab is depicted as a heroic figure due to his exceptional skills as a whale hunter and his strong leadership abilities. The stranger\'s description of Ahab as a "good whale-hunter, and a good captain to his crew" highlights his professional competence and his ability to command respect and loyalty from his crew members. Furthermore, Ahab\'s physical and mental strength, as well as his unconventional and independent thinking, contribute to his heroic image. The document\'s reference to him as a "man of greatly superior natural force, with a globular brain and a ponderous heart" underscores his exceptional qualities and his potential to achieve great things.\n\nOn the other hand, Ahab is also portrayed as a villainous character due to his morbid obsession with hunting down Moby Dick, the white whale that ha